In [7]:
import tkinter as tk
from tkinter import messagebox
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Load data
df_credits = pd.read_csv("tmdb_5000_credits.csv")
df_movies = pd.read_csv("tmdb_5000_movies.csv")

# TF-IDF Vectorization
tfidf = TfidfVectorizer(stop_words="english")
df_movies['overview'] = df_movies['overview'].fillna("")
tfidf_matrix = tfidf.fit_transform(df_movies['overview'])

# Compute cosine similarity
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Create indices Series
indices = pd.Series(df_movies.index, index=df_movies['original_title']).drop_duplicates()

# Function to get recommendations
def get_recommendations(movie_title):
    if movie_title.strip() == "":
        return "Please enter a movie name."
    
    if movie_title in indices:
        idx = indices[movie_title]
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:11]  # Get top 10 similar movies
        
        sim_indices = [i[0] for i in sim_scores]
        recommended_movies = df_movies['original_title'].iloc[sim_indices]
        return recommended_movies
    else:
        return "Movie not found in the database."

# Function to handle button click
def on_button_click():
    movie_title = entry.get()
    result_text.set(get_recommendations(movie_title))

# Create main window
root = tk.Tk()
root.title("Movie Recommendation System")

# Styling
root.geometry("500x400")  # Set window size
root.configure(background="black")  # Set background color

# Create labels, entry, button, and result label
label = tk.Label(root, text="Enter a Movie Name:", font=("Helvetica", 14), bg="black", fg="white")
label.pack(pady=10)  # Add padding

entry = tk.Entry(root, font=("Helvetica", 12))
entry.pack(pady=5)  # Add padding

button = tk.Button(root, text="Get Recommendations", command=on_button_click, font=("Georgia", 12), bg="Red", fg="white")
button.pack(pady=10)  # Add padding

tk.Label(root, text="", bg="black").pack(pady=5) #empty space

label = tk.Label(root, text="Recommendations", font=("Helvetica", 14), bg="beige", fg="black")
label.pack(pady=10)  # Add padding
    
result_text = tk.StringVar()
result_label = tk.Label(root, textvariable=result_text, wraplength=480, justify="left", font=("Helvetica", 12), bg="beige", fg="black")
result_label.pack(pady=10)  # Add padding

# Run the application
root.mainloop()
